In [ ]:
%pip install transformers
%pip install datasets
%pip install huggingface_hub
%pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import torch
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# Load your dataset
df = pd.read_excel('/content/intent_classification_train.xlsx')

intent_label_dict = {0: 'Divorce', 1: 'Consumer protection',2: 'Inheritance', 3: 'Copyright', 4: 'Other'}

df.replace({'Divorce':0, 'Consumer protection':1, 'Inheritance':2, 'Copyright':3, 'Other':4}, inplace = True)

X = df['Query'].copy()
y = df['Intent'].copy()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

train_df = pd.concat([X_train, y_train], axis = 1)
test_df = pd.concat([X_test, y_test], axis = 1)

# Convert to Hugging Face Dataset format
raw_datasets = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "eval": Dataset.from_pandas(test_df)
})

<ipython-input-3-d20f2cc7ebf7>:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'Divorce':0, 'Consumer protection':1, 'Inheritance':2, 'Copyright':3, 'Other':4}, inplace = True)


In [ ]:
train_df['Intent'].value_counts()

,count
Intent,
0,40
2,40
4,40
3,40
1,39


In [ ]:
test_df['Intent'].value_counts()

,count
Intent,
2,10
3,10
4,10
1,10
0,10


In [ ]:
checkpoint = "bert-base-uncased"
classifier = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5) # Get the classifier
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokenized_datasets = raw_datasets.map(lambda dataset: tokenizer(dataset['Query'], truncation=True), batched=True)
print(tokenized_datasets)

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Query', 'Intent', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 199
    })
    eval: Dataset({
        features: ['Query', 'Intent', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50
    })
})


In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["Query", "__index_level_0__"])
tokenized_datasets = tokenized_datasets.rename_column("Intent", "labels")
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 199
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50
    })
})


In [ ]:
# Padding for batch of data that will be fed into model for training
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training args
training_args = TrainingArguments("test-trainer", num_train_epochs=5, evaluation_strategy="epoch",
                                  weight_decay=5e-4, save_strategy="no", report_to="none")

# Define a compute_metrics function for multi-class classification
def compute_metrics(p):
    # Extract the predictions
    preds = p.predictions.argmax(-1)  # Get the index of the maximum logit value for each example
    labels = p.label_ids

    # Compute accuracy
    accuracy = accuracy_score(labels, preds)

    # Compute precision, recall, f1 for each class (weighted average)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define trainer
trainer = Trainer(
    classifier,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-b2027d09a8eb>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.114489,0.520000,0.562500,0.520000,0.480159
2,No log,0.392878,0.940000,0.948485,0.940000,0.939546
3,No log,0.076071,1.000000,1.000000,1.000000,1.000000
4,No log,0.035186,1.000000,1.000000,1.000000,1.000000
5,No log,0.027015,1.000000,1.000000,1.000000,1.000000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=125, training_loss=0.485419677734375, metrics={'train_runtime': 354.4347, 'train_samples_per_second': 2.807, 'train_steps_per_second': 0.353, 'total_flos': 7310778700572.0, 'train_loss': 0.485419677734375, 'epoch': 5.0})

In [ ]:
metrics_train = trainer.evaluate(eval_dataset=tokenized_datasets["train"])
print("train Set Metrics:", metrics_train)

train Set Metrics: {'eval_loss': 0.014612034894526005, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 23.866, 'eval_samples_per_second': 8.338, 'eval_steps_per_second': 1.048, 'epoch': 5.0}


In [ ]:
metrics_test = trainer.evaluate(eval_dataset=tokenized_datasets["eval"])
print("Test Set Metrics:", metrics_test)

Test Set Metrics: {'eval_loss': 0.027015123516321182, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 5.0008, 'eval_samples_per_second': 9.998, 'eval_steps_per_second': 1.4, 'epoch': 5.0}
